In [1]:
!BioDendro -h

usage: BioDendro [-h] [-n] [-c CUTOFF] [-b BIN_THRESHOLD]
                 [-d {jaccard,braycurtis}] [-m MATRIX] [-p PROCESSED]
                 [-o OUT_HTML] [-r RESULTS_DIR] [-x WIDTH_PX] [-y HEIGHT_PX]
                 mgf components

Process MGF file into appropriate input for biodendro.

positional arguments:
  mgf                   MGF input file (file1.mgf)
  components            Listed components file (file2.txt)

optional arguments:
  -h, --help            show this help message and exit
  -n, --neutral         Apply neutral loss.
  -c CUTOFF, --cutoff CUTOFF
  -b BIN_THRESHOLD, --bin-threshold BIN_THRESHOLD
  -d {jaccard,braycurtis}, --cluster-method {jaccard,braycurtis}
  -m MATRIX, --matrix MATRIX
  -p PROCESSED, --processed PROCESSED
                        Path to write output to.
  -o OUT_HTML, --output OUT_HTML
                        Path to write output to.
  -r RESULTS_DIR, --results-dir RESULTS_DIR
  -x WIDTH_PX, --width WIDTH_PX
  -y HEIGHT_PX, --height HEIGHT_PX


In [2]:
%%bash
rm -rf -- ./results
BioDendro --processed processed.xlsx ./MSMS.mgf ./component_list.txt

Please be patient..It may take a while to compute..


In [1]:
from importlib import reload
import plotly
import pandas as pd
import BioDendro.plot

reload(BioDendro.plot)

from BioDendro.plot import Dendrogram


In [2]:
data = Dendrogram.from_xlsx("./processed.xlsx")

In [3]:
data.clusterize()

In [4]:
data.generate_linkage()

Please be patient..It may take a while to compute..


In [5]:
data.generate_out()

In [6]:
data.mycluster

array([117,  28, 119,  92,  33,  38,  28, 125,  20,  81,  85,  57,  16,
        25,  74,  22,  24, 101,  85,  86,  75, 104,  31,  19,  15, 113,
        34,  25,  23,  24,   1,  15,  21,  28, 119, 115,  33,   8,   9,
         8,  23,   8, 116,  17,  28,  23, 112, 102, 120,  55,  91, 120,
       118,  43,  53, 105,  96,  27,  96,  33,  26,  18,  23,  18,  26,
        21,  64,  63,  16,  22,  29, 109,  44,  13,  28,  28,  36, 107,
        31,  28,  33, 107, 114,  28,  47,  35,  28,  28, 110,  28,  62,
        33, 111,  31,  30,  28,  16,  44,  58,   5,  79, 120,  87, 119,
       120, 120, 120, 120, 120, 120, 122, 123, 121, 122, 123, 124, 106,
        91,  68,  73, 121, 118,  83,  59,  60,  59,  57,  11,  69,  67,
        92,  32,  67, 108,  70,  37,  94,  50,  97,  67,  99,  54, 103,
        98, 105, 104,  82,  84, 100,  95,  41, 126,   2,  45,  91,  91,
        91,  10,   3,   4,   6,   7,  89,  66,   7,  93,  88,  86,  90,
        77,  80,  42,  39,  13,  40,  49,  14,  72,  13,  71,  7

In [7]:
k = data.visualize(cutoff=0.5)

In [11]:
plotly.offline.init_notebook_mode(connected=True) # for visualising plot inline
plotly.offline.iplot(k, filename='simple_dendrogram')

NameError: name 'Dendogram' is not defined